In [23]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from skimage import io

In [24]:
# Function to load images from folders
def load_images_from_folder(path):
    images = []
    for folder in path:
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            if os.path.isfile(img_path):
                image = io.imread(img_path, as_gray=True)
                images.append(image.flatten())
    return np.array(images)

In [26]:
# Training data
angry_folder = r'C:\Users\hp\Desktop\archive\train\angry'  # Path to the folder containing angry images
disgust_folder = r'C:\Users\hp\Desktop\archive\train\disgust'  # Path to the folder containing disgust images
fear_folder = r'C:\Users\hp\Desktop\archive\train\fear'  # Path to the folder containing fear images
happy_folder = r'C:\Users\hp\Desktop\archive\train\happy'  # Path to the folder containing happy images
neutral_folder = r'C:\Users\hp\Desktop\archive\train\neutral'  # Path to the folder containing neutral images
sad_folder = r'C:\Users\hp\Desktop\archive\train\sad'  # Path to the folder containing sad images
surprise_folder = r'C:\Users\hp\Desktop\archive\train\surprise'  # Path to the folder containing surprise images

train_data_angry = load_images_from_folder([angry_folder])
train_data_disgust = load_images_from_folder([disgust_folder])
train_data_fear = load_images_from_folder([fear_folder])
train_data_happy = load_images_from_folder([happy_folder])
train_data_neutral = load_images_from_folder([neutral_folder])
train_data_sad = load_images_from_folder([sad_folder])
train_data_surprise = load_images_from_folder([surprise_folder])

# Combine and label the training data
train_data = np.vstack((train_data_angry, train_data_disgust, train_data_fear,
                        train_data_happy, train_data_neutral, train_data_sad, train_data_surprise))


In [27]:
num_clusters = 7

In [28]:
m = 2  # Fuzziness coefficient
tolerance = 1e-6  # Convergence threshold
num_epochs = 10000
dropout_rate = 0.2  # Dropout rate for regularization

# Initialize the membership matrix
num_samples = len(train_data)
membership = np.random.rand(num_samples, num_clusters)
membership = membership / np.sum(membership, axis=1)[:, np.newaxis]

# Fuzzy C-means algorithm
for epoch in range(num_epochs):
    # Compute the cluster centers
    cluster_centers = np.dot(membership.T, train_data) / np.sum(membership, axis=0)[:, np.newaxis]

    # Compute the distances between samples and cluster centers
    distances = np.linalg.norm(train_data[:, np.newaxis] - cluster_centers, axis=2)

    # Update the membership matrix
    new_membership = 1 / (distances ** (2 / (m - 1)))
    new_membership = new_membership / np.sum(new_membership, axis=1)[:, np.newaxis]

    # Check for convergence
    if np.max(np.abs(new_membership - membership)) < tolerance:
        break

    membership = new_membership

# Assign labels based on the highest membership value
predicted_labels = np.argmax(membership, axis=1)

# Convert the labels to TensorFlow tensor
train_labels = tf.convert_to_tensor(predicted_labels, dtype=tf.int64)

# Define the LVQ model
class LVQ(keras.Model):
    def __init__(self, num_classes, input_dim, prototypes_per_class, dropout_rate):
        super(LVQ, self).__init__()
        self.num_classes = num_classes
        self.input_dim = input_dim
        self.prototypes_per_class = prototypes_per_class

        self.prototypes = self.add_weight(shape=(num_classes * prototypes_per_class, input_dim),
                                          initializer='random_normal',
                                          trainable=False)  # Freeze the prototypes during training
        self.dropout = keras.layers.Dropout(dropout_rate)  # Add dropout layer

    def call(self, x):
        # Compute distances between input samples and prototypes
        distances = tf.norm(x[:, tf.newaxis] - self.prototypes, axis=-1)

        # Get the nearest prototypes for each input sample
        nearest_prototypes = tf.argmin(distances, axis=-1)

        return nearest_prototypes


# Convert the data to TensorFlow tensors
train_data = tf.convert_to_tensor(train_data, dtype=tf.float32)

# Create an instance of the LVQ model
#model = LVQ(num_clusters, train_data.shape[1], 1)
model = LVQ(num_clusters, train_data.shape[1], 1, dropout_rate)

# Define the loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# Training loop
for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        # Forward pass
        outputs = model(train_data)
        # Convert the outputs to one-hot encoded logits
        logits = tf.one_hot(outputs, depth=num_clusters)
        # Compute the loss
        loss = loss_fn(train_labels, logits)
    # Compute gradients
    grads = tape.gradient(loss, model.trainable_variables)
    # Update weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    # Print the loss for monitoring
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}")

Epoch [10/10000], Loss: 1.6676
Epoch [20/10000], Loss: 1.6676
Epoch [30/10000], Loss: 1.6676
Epoch [40/10000], Loss: 1.6676
Epoch [50/10000], Loss: 1.6676
Epoch [60/10000], Loss: 1.6676
Epoch [70/10000], Loss: 1.6676
Epoch [80/10000], Loss: 1.6676
Epoch [90/10000], Loss: 1.6676
Epoch [100/10000], Loss: 1.6676
Epoch [110/10000], Loss: 1.6676
Epoch [120/10000], Loss: 1.6676
Epoch [130/10000], Loss: 1.6676
Epoch [140/10000], Loss: 1.6676
Epoch [150/10000], Loss: 1.6676
Epoch [160/10000], Loss: 1.6676
Epoch [170/10000], Loss: 1.6676
Epoch [180/10000], Loss: 1.6676
Epoch [190/10000], Loss: 1.6676
Epoch [200/10000], Loss: 1.6676
Epoch [210/10000], Loss: 1.6676
Epoch [220/10000], Loss: 1.6676
Epoch [230/10000], Loss: 1.6676
Epoch [240/10000], Loss: 1.6676
Epoch [250/10000], Loss: 1.6676
Epoch [260/10000], Loss: 1.6676
Epoch [270/10000], Loss: 1.6676
Epoch [280/10000], Loss: 1.6676
Epoch [290/10000], Loss: 1.6676
Epoch [300/10000], Loss: 1.6676
Epoch [310/10000], Loss: 1.6676
Epoch [320/10000]

Epoch [2530/10000], Loss: 1.6676
Epoch [2540/10000], Loss: 1.6676
Epoch [2550/10000], Loss: 1.6676
Epoch [2560/10000], Loss: 1.6676
Epoch [2570/10000], Loss: 1.6676
Epoch [2580/10000], Loss: 1.6676
Epoch [2590/10000], Loss: 1.6676
Epoch [2600/10000], Loss: 1.6676
Epoch [2610/10000], Loss: 1.6676
Epoch [2620/10000], Loss: 1.6676
Epoch [2630/10000], Loss: 1.6676
Epoch [2640/10000], Loss: 1.6676
Epoch [2650/10000], Loss: 1.6676
Epoch [2660/10000], Loss: 1.6676
Epoch [2670/10000], Loss: 1.6676
Epoch [2680/10000], Loss: 1.6676
Epoch [2690/10000], Loss: 1.6676
Epoch [2700/10000], Loss: 1.6676
Epoch [2710/10000], Loss: 1.6676
Epoch [2720/10000], Loss: 1.6676
Epoch [2730/10000], Loss: 1.6676
Epoch [2740/10000], Loss: 1.6676
Epoch [2750/10000], Loss: 1.6676
Epoch [2760/10000], Loss: 1.6676
Epoch [2770/10000], Loss: 1.6676
Epoch [2780/10000], Loss: 1.6676
Epoch [2790/10000], Loss: 1.6676
Epoch [2800/10000], Loss: 1.6676
Epoch [2810/10000], Loss: 1.6676
Epoch [2820/10000], Loss: 1.6676
Epoch [283

Epoch [5020/10000], Loss: 1.6676
Epoch [5030/10000], Loss: 1.6676
Epoch [5040/10000], Loss: 1.6676
Epoch [5050/10000], Loss: 1.6676
Epoch [5060/10000], Loss: 1.6676
Epoch [5070/10000], Loss: 1.6676
Epoch [5080/10000], Loss: 1.6676
Epoch [5090/10000], Loss: 1.6676
Epoch [5100/10000], Loss: 1.6676
Epoch [5110/10000], Loss: 1.6676
Epoch [5120/10000], Loss: 1.6676
Epoch [5130/10000], Loss: 1.6676
Epoch [5140/10000], Loss: 1.6676
Epoch [5150/10000], Loss: 1.6676
Epoch [5160/10000], Loss: 1.6676
Epoch [5170/10000], Loss: 1.6676
Epoch [5180/10000], Loss: 1.6676
Epoch [5190/10000], Loss: 1.6676
Epoch [5200/10000], Loss: 1.6676
Epoch [5210/10000], Loss: 1.6676
Epoch [5220/10000], Loss: 1.6676
Epoch [5230/10000], Loss: 1.6676
Epoch [5240/10000], Loss: 1.6676
Epoch [5250/10000], Loss: 1.6676
Epoch [5260/10000], Loss: 1.6676
Epoch [5270/10000], Loss: 1.6676
Epoch [5280/10000], Loss: 1.6676
Epoch [5290/10000], Loss: 1.6676
Epoch [5300/10000], Loss: 1.6676
Epoch [5310/10000], Loss: 1.6676
Epoch [532

Epoch [7510/10000], Loss: 1.6676
Epoch [7520/10000], Loss: 1.6676
Epoch [7530/10000], Loss: 1.6676
Epoch [7540/10000], Loss: 1.6676
Epoch [7550/10000], Loss: 1.6676
Epoch [7560/10000], Loss: 1.6676
Epoch [7570/10000], Loss: 1.6676
Epoch [7580/10000], Loss: 1.6676
Epoch [7590/10000], Loss: 1.6676
Epoch [7600/10000], Loss: 1.6676
Epoch [7610/10000], Loss: 1.6676
Epoch [7620/10000], Loss: 1.6676
Epoch [7630/10000], Loss: 1.6676
Epoch [7640/10000], Loss: 1.6676
Epoch [7650/10000], Loss: 1.6676
Epoch [7660/10000], Loss: 1.6676
Epoch [7670/10000], Loss: 1.6676
Epoch [7680/10000], Loss: 1.6676
Epoch [7690/10000], Loss: 1.6676
Epoch [7700/10000], Loss: 1.6676
Epoch [7710/10000], Loss: 1.6676
Epoch [7720/10000], Loss: 1.6676
Epoch [7730/10000], Loss: 1.6676
Epoch [7740/10000], Loss: 1.6676
Epoch [7750/10000], Loss: 1.6676
Epoch [7760/10000], Loss: 1.6676
Epoch [7770/10000], Loss: 1.6676
Epoch [7780/10000], Loss: 1.6676
Epoch [7790/10000], Loss: 1.6676
Epoch [7800/10000], Loss: 1.6676
Epoch [781

Epoch [10000/10000], Loss: 1.6676


In [20]:
# Test with a new image
new_image_path = "C:\\Users\\hp\\Desktop\\archive\\test\\sad\\PrivateTest_366361.jpg"  # Path to the new image file
if os.path.isfile(new_image_path):
    new_image = io.imread(new_image_path, as_gray=True)
    new_image = np.array(new_image.flatten())
    new_image = tf.convert_to_tensor(new_image, dtype=tf.float32)
    new_image = tf.expand_dims(new_image, axis=0)  # Add a batch dimension

    predicted_label = model(new_image)
    predicted_class = "Happy" if predicted_label.numpy().item() == 0 else "Sad"

    print("Predicted emotion:", predicted_class)

Predicted emotion: Sad


In [21]:
# Test with a new image
new_image_path = "C:\\Users\\hp\\Desktop\\archive\\test\\surprise\\PrivateTest_2017514.jpg"  # Path to the new image file
if os.path.isfile(new_image_path):
    new_image = io.imread(new_image_path, as_gray=True)
    new_image = np.array(new_image.flatten())
    new_image = tf.convert_to_tensor(new_image, dtype=tf.float32)
    new_image = tf.expand_dims(new_image, axis=0)  # Add a batch dimension

    predicted_label = model(new_image)
    predicted_class = "Happy" if predicted_label.numpy().item() == 0 else "Sad"

    print("Predicted emotion:", predicted_class)

Predicted emotion: Sad


In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from skimage import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Function to load images from folders
def load_images_from_folder(folders):
    images = []
    for folder in folders:
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            if os.path.isfile(img_path):
                image = io.imread(img_path, as_gray=True)
                images.append(image.flatten())
    return np.array(images)

# Normalize the input data
def normalize_data(data):
    scaler = StandardScaler()
    data_normalized = scaler.fit_transform(data)
    return data_normalized

# Training data
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
train_data_folders = [f'C:\\Users\\hp\\Desktop\\archive\\train\\{emotion}' for emotion in emotions]
train_data = load_images_from_folder(train_data_folders)

# Normalize the training data
train_data = normalize_data(train_data)

# Labels for training data
train_labels = np.concatenate([np.full(len(data), i) for i, data in enumerate(train_data_folders)])

# Convert the labels to TensorFlow tensor
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int64)

# Shuffle the data and labels
perm = np.random.permutation(len(train_labels))
train_data = train_data[perm]
train_labels = tf.gather(train_labels, perm)

# Convert train_labels to NumPy array
train_labels = train_labels.numpy()

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Hyperparameters
num_clusters = len(emotions)
m = 2  # Fuzziness coefficient
tolerance = 1e-6  # Convergence threshold
num_epochs = 10000
dropout_rate = 0.2  # Dropout rate for regularization
learning_rate = 0.001

# Initialize the membership matrix
num_samples = len(train_data)
membership = np.random.rand(num_samples, num_clusters)
membership = membership / np.sum(membership, axis=1)[:, np.newaxis]

# Fuzzy C-means algorithm
for epoch in range(num_epochs):
    # Compute the cluster centers
    cluster_centers = np.dot(membership.T, train_data) / np.sum(membership, axis=0)[:, np.newaxis]

    # Compute the distances between samples and cluster centers
    distances = np.linalg.norm(train_data[:, np.newaxis] - cluster_centers, axis=2)

    # Update the membership matrix
    new_membership = 1 / (distances ** (2 / (m - 1)))
    new_membership = new_membership / np.sum(new_membership, axis=1)[:, np.newaxis]

    # Check for convergence
    if np.max(np.abs(new_membership - membership)) < tolerance:
        break

    membership = new_membership

# Assign labels based on the highest membership value
predicted_labels = np.argmax(membership, axis=1)

# Convert the labels to TensorFlow tensor
predicted_labels = tf.convert_to_tensor(predicted_labels, dtype=tf.int64)

# Evaluate the accuracy
accuracy = np.mean(predicted_labels == train_labels)
print(f"Accuracy: {accuracy}")


Accuracy: 0.18099547511312217


# metrics

In [36]:
happy_train_path="C:\\Users\\hp\\Desktop\\archive\\train\\happy"
sad_train_path="C:\\Users\\hp\\Desktop\\archive\\train\\sad"
surprised_train_path = "C:\\Users\\hp\\Desktop\\archive\\train\\surprise"

# Function to load images from folders
def load_images_from_folder(path):
    images = []
    for folder in path:
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            if os.path.isfile(img_path):
                image = io.imread(img_path, as_gray=True)
                images.append(image.flatten() / 255.)
    return np.array(images)

# Training data
train_data_happy = load_images_from_folder([happy_train_path])
train_data_sad = load_images_from_folder([sad_train_path])
train_data_surprised = load_images_from_folder([surprised_train_path])

In [37]:
train_data = np.vstack((train_data_happy, train_data_sad, train_data_surprised))

In [38]:
ha = [0 for e in range(train_data_happy.shape[0])]
sa = [1 for e in range(train_data_sad.shape[0])]
su = [2 for e in range(train_data_surprised.shape[0])]
train_labels = ha + sa + su

In [42]:
D = train_labels

import random as rd
class FCM:
    def __init__(self, m, c, X):
        self.m = m
        self.c = c
        self.p = X.shape[1]
        minX = [min(X[:,i]) for i in range(X.shape[1])]
        maxX = [max(X[:,i]) for i in range(X.shape[1])]
        self.V = np.array([ [ rd.uniform(minX[j], maxX[j]) for j in range(X.shape[1])] for i in range(self.c)])
        self.U = []
        self.Etqt = []
    def calculerU(self, X):
        alpha = -1./(self.m-1.)
        term1 = [ [ pow(sum(np.square(X[k]-self.V[i])), alpha) for k in range(X.shape[0]) ] for i in range(self.c)]
        term2 = [ sum([pow(sum(np.square(X[k]-self.V[i])), alpha) for i in range(self.c)]) for k in range(X.shape[0])]
        self.U = [ [ term1[i][k]/term2[k] for k in range(X.shape[0])] for i in range(self.c)]
    def calculerV(self, X):
        card = [ sum([pow(self.U[i][k], self.m) for k in range(X.shape[0])]) for i in range(self.c)]
        #print(sum(card)) #à cause de la puissance m, la somme est différente de n
        term = [ [ sum([ pow(self.U[i][k], self.m)*X[k][j] for k in range(X.shape[0])]) for j in range(X.shape[1])] for i in range(self.c)]
        self.V = [ [ term[i][j]/card[i] for j in range(X.shape[1])] for i in range(self.c)]
    def entrainer(self, X, tmax, eps):
        t = 1
        while(t<tmax):
            print(t)
            self.calculerU(X)
            self.calculerV(X)
            t = t + 1
            self.etiqueterV(X)
            #print(self.tauxErr(X, D)*100./X.shape[0],"%")
    def etiqueterV(self, X):
        card = [0 for i in range(self.c)]
        Vmoy = [ [ 0. for j in range(X.shape[1])] for i in range(self.c)]
        for k in range(X.shape[0]):
            for j in range(X.shape[1]):
                Vmoy[D[k]-1][j] = Vmoy[D[k]-1][j] + X[k][j]
        for k in range(X.shape[0]):
            card[D[k]-1] = card[D[k]-1] + 1
        for i in range(self.c):
            for j in range(X.shape[1]):
                Vmoy[i][j] = Vmoy[i][j] / card[D[k]-1]
        Vmoy = np.array(Vmoy)
        #print("Vmoy = \n"+str(np.array(Vmoy)))
        dist = [[sum(np.square(self.V[l]-Vmoy[i])) for i in range(self.c)] for l in range(self.c)]
        etqt = [np.argmin(dist[i]) for i in range(self.c)]
        for i in range(self.c):
            self.Etqt.append(etqt[i]+1)
        #print(np.array(self.Etqt))i
    def classe(self, vect):
        term1 = [ pow(sum(np.square(vect-self.V[i])), -1./(self.m-1)) for i in range(self.c)]
        term2 = sum([ pow(sum(np.square(vect-self.V[i])), -1./(self.m-1)) for i in range(self.c)])
        u = [term1[i]/term2 for i in range(self.c)]
        return self.Etqt[np.argmax(u)]

    def tauxErr(self, X, D):
        er = 0.
        for k in range(X.shape[0]):
            if(self.classe(X[k]) != D[k]):
                er = er + 1
        return er
    
    def entropie(self,X):
        En = []
        En = sum(sum(  self.U[i][k]*(math.log2(self.U[i][k]) ) for i in range(self.c)) for k in range(X.shape[0] ))
        return -En/X.shape[0]
    
    def pc(self, X):
        xp = sum(sum(np.square(fcm.U[i][k]) for i in range(c)) for k in range(X.shape[0]))
        return xp/X.shape[0]

    def matriceConfusion(self,X, D):
        matConf = np.zeros((fcm.c, fcm.c), dtype=int)
        for i in range(X.shape[0]):
            vraieClasse = D[i]-1
            predClasse = fcm.classe(X[i])-1
            matConf[vraieClasse, predClasse] += 1
        return matConf

In [ ]:
from sklearn.metrics import confusion_matrix

fcm = FCM(2, 3, train_data)
fcm.calculerU(train_data)
#for i in range(X.shape[0]):
#print(np.array(fcm.U)[:,i])
fcm.calculerV(train_data)
print("les prototypes avant l'entrainement :\n",np.array(fcm.V))
print()
fcm.entrainer(train_data, 1000, 0.001)
print("les prototypes apres l'entrainement :\n",np.array(fcm.V))
# Créer la matrice de confusion en utilisant scikit-learn
confusion = confusion_matrix(train_labels, fcm.predict(train_data))
print("la matrice de confusion est :\n",confusion)
#print(fcm.tauxErr(X, D)*100./X.shape[0],"%")

les prototypes avant l'entrainement :
 [[0.48058255 0.46935146 0.45730313 ... 0.44635354 0.44972837 0.45253919]
 [0.48349621 0.4725199  0.46071125 ... 0.44030045 0.44379847 0.44675902]
 [0.47732284 0.46585027 0.45375107 ... 0.44520167 0.44841604 0.45117118]]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
